In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 先载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

# 输入图片28*28
n_inputs = 28 #输入一行，一行28个数据
max_time = 28 #共28行
lstm_size = 100 #基础单元
n_classes = 10 #10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples // batch_size #多少个批次

# 两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

# 初始化权值,100*10
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
# 初始化偏置,10
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

# 定义RNN网络
def RNN(X,weights,biases):
    # input = [batch_size,max_time,n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    # 定义LSTM基本cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
#     initial_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    outputs,state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    
    results = tf.nn.softmax(tf.matmul(state[1],weights) + biases)
    return results

# 预测值
prediction = RNN(x,weights,biases)
# 损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
# Opitmizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放，布尔值类型
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
# Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
# init
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter " + str(epoch) + ",Testing Accuracy: " + str(acc))
        
        

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter 0,Testing Accuracy: 0.8033
Iter 1,Testing Accuracy: 0.8914
Iter 2,Testing Accuracy: 0.9153
Iter 3,Testing Accuracy: 0.9282
Iter 4,Testing Accuracy: 0.9279
Iter 5,Testing Accuracy: 0.9382


> [TensorFlow中RNN实现的正确打开方式](https://zhuanlan.zhihu.com/p/28196873)

>  详细解释RNN的使用方法以及tf.nn.dynamic_rnn(),是一个系统的学习流程，作者很优秀。

> 不知道为什么，在53行加了个initial_state就会报错。网上查到是tf.nn.dynamic_rnn()移除了initial这个参数。

> 还有一个就是我跑的比原版本慢许多，不知道原因。

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
WARNING:tensorflow:From <ipython-input-2-353d081bc6d2>:36: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter 0,Testing Accuracy: 0.8033
Iter 1,Testing Accuracy: 0.8914
Iter 2,Testing Accuracy: 0.9153
Iter 3,Testing Accuracy: 0.9282
Iter 4,Testing Accuracy: 0.9279
Iter 5,Testing Accuracy: 0.9382